In [21]:
!pip install simplemma

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.5/75.5 MB 7.7 MB/s eta 0:00:00


In [22]:
from simplemma import text_lemmatizer
import pandas as pd
import numpy as np
import ast
import re

In [23]:
df = pd.read_csv("/content/review_info.csv")
languages = ["turkish"]
df = df[df.language.isin(languages)].copy() #sadece turkce olan yorumlari alip kayit edecek
df.to_excel('turkce_veriler.xlsx', index=False) # bu dosyaya kayıt edecek
df.head()

,review_id,language,review,timestamp_created,timestamp_updated,recommended,votes_helpful,votes_funny,weighted_vote_score,comment_count
216,68574648,turkish,muhtesem bir oyun.,1588623915,1588623915,True,0,0,0.00000,0
243,68562271,turkish,"Oynamaya doymadığım, tekrar tekrar baştan başl...",1588608197,1588608197,True,1,0,0.52381,0
406,68493860,turkish,mükemmel,1588511900,1588511900,True,0,0,0.00000,0
410,68492010,turkish,Karakterimi yeni bitirdim.,1588509720,1588509720,True,0,1,0.00000,0
589,68401494,turkish,Dünyalardan Dünya beğen,1588384424,1588384424,True,2,0,0.52381,0


In [25]:
stop_words = None
with open("stopwords.txt", "r") as stop_file:
    stop_words = set(stop_file.read().splitlines())

In [37]:
def clean_text(text):
    text = text.replace("Â", "a")
    text = text.replace("â", "a")
    text = text.replace("î", "i")
    text = text.replace("Î", "ı")
    text = text.replace("İ", "i")
    text = text.replace("I", "ı")
    text = text.replace(u"\u00A0", " ")
    text = text.replace("|", " ")

    text = re.sub(r"@[A-Za-z0-9]+", " ", text)
    text = re.sub(r"(.)\1+", r"\1\1", text)
    text = re.sub(r"https?:\/\/\S+", " ", text)
    text = re.sub(r"http?:\/\/\S+", " ", text)
    text = re.sub(r"\n", " ", text)
    text = re.sub(r"#(\w+)", " ", text)
    text = re.sub(r"^\x00-\x7F]+", " ", text)
    text = re.sub(r"[^A-Za-zâîığüşöçİĞÜŞÖÇ]+", " ", text)
    text = re.sub(r"((https://[^\s]+))", " ", text)

    text = " ".join(text.lower().strip().split())
    text = text_lemmatizer(text, lang="tr")

    return " ".join([word for word in text if word not in stop_words])

In [27]:
df = pd.read_excel("/content/turkce_veriler.xlsx")
df.head()

,review_id,language,review,timestamp_created,timestamp_updated,recommended,votes_helpful,votes_funny,weighted_vote_score,comment_count
0,68574648,turkish,muhtesem bir oyun.,1588623915,1588623915,True,0,0,0.00000,0
1,68562271,turkish,"Oynamaya doymadığım, tekrar tekrar baştan başl...",1588608197,1588608197,True,1,0,0.52381,0
2,68493860,turkish,mükemmel,1588511900,1588511900,True,0,0,0.00000,0
3,68492010,turkish,Karakterimi yeni bitirdim.,1588509720,1588509720,True,0,1,0.00000,0
4,68401494,turkish,Dünyalardan Dünya beğen,1588384424,1588384424,True,2,0,0.52381,0


In [28]:
len(df)

21320

In [42]:
df["review"] = df["review"].astype(str) # AttributeError: 'float' object has no attribute 'replace' hatasi için
df["clean"] = df["review"].apply(clean_text)

In [43]:
df.to_excel('cleandata.xlsx', index=False)

In [44]:
import random
for i in range(3):
    rand_idx = random.randrange(len(df))
    ozet = df.iloc[rand_idx]["review"]
    print("Orijinal:")
    print(ozet + "\n")
    print("Temizlenmiş:")
    print(df.iloc[rand_idx]["clean"])
    print("/" * 50 + "\n")

Orijinal:
şu oyuna 179 okdar cokkı bence 20 lırayı bıle haketmıyor okdar sacma kı yapay zeka adam mac boyu her defasında aynı adam gelıyor 1 defa ben gelıyorum 20 defa adamlar 1atakta bır gol atıp oyunu kazanıyor ıstedgın kadar savunma taktık yap yıne gol yıyorsun herseyo yap dene olmuyor senın forvetın bıtırıcılıgı 19 karsınınkı 10 adam gelıp sana gol atıyor sana 10 kere pzoıyon gelıyor forvetıne atamıyor tamamen sacmalık abı

Temizlenmiş:
oyun okdar cokkı lırayı bıle haketmıyor okdar sacma kı yapay zeka adam mac boy aynı adam gelıyor gelıyorum adam atak bır gol at oyun kazan ıstedgın savun tak yıne gol yıyorsun herseyo dene senın forvetın bıtırıcılıgı kars adam gelıp san gol at san kere pzoıyon gelıyor forvetıne at sacmalık ab
//////////////////////////////////////////////////

Orijinal:
Gayet güzel grafiklenmiş çocukluğumun oyunlarından biridir bu tarz oyunları sevenler için mükemmel bir seçim tavsiye ederim.

Temizlenmiş:
gayet güzel grafiklenmiş çocukluk oyun tarz oyun sev mükemme